# Loan Pricing #
<hr class="header-line"/>

Pricing of a loan, just like any other product, is influenced by both the cost to produce and the market.

<p class="p-break"></p>

And so there are two methods to deriving an interest rate for a loan:

1. Capital-Driven:
    + what interest rate is required to justify the allocation of capital?

2. Market-Driven:
    + what rate is implied in the market?
    + derived from various market-related exposures
    
The two methods are *not* exclusive. Fico should consider both approaches when loan pricing as circumstances may reveal information:

<p class="p-break"></p>

$\text{Market-Driven Rate} >>> \text{Capital Driven Rate}$

+ Fico either has some competitive advantage *or* it is underpricing risk

<p class="p-break"></p>

$\text{Capital Driven Rate} >>> \text{Market-Driven Rate}$

+ Fico has a competitive disadvantage *or* it is overpricing risk

<p class="p-break"></p>
<p class="p-break"></p>

## Capital Driven ##

+ Loan income is compared to a *hurdle rate*, which can be set as the Weighted Average Cost of Capital (WACC) for the company/investor/project.

$$
\text{WACC} = r_{tgt} = w_{debt}*r_{debt} + w_{equity}*r_{equity} 
$$
$\qquad\qquad\qquad\qquad\qquad\textit{where:}$
$$
\\w_{debt} = \frac{D}{D+E}
$$

$\qquad$or simply as the cost of equity, to take the investor perspective:
   
$$ r_{tgt} = r_{equity} $$

$\qquad$ The hurdle rate may also be set relative to other investment opportunities (and thereby seen as an opportunity cost).

<p class="p-break"></p>

+ The interest rate required *for an individual loan* to meet the hurdle rate is:

$$
\begin{alignat*}{0}
\qquad\text{int}_i = \frac{r_{tgt}*(D_i+E_i) + exp_i}{EAD_i}
\\\textit{where:}
\\\qquad exp_i = \text{expected losses + non-recovery related expenses like salaries, taxes, etc.}
\\\qquad D_i, E_i = \text{debt and equity contribution required for the individual loan}
\\\qquad D_i + E_i = EAD_i
\end{alignat*}
$$



**Equity Only**

We will estimate pricing for loans where Fico is funded entirely by equity.

<p class="p-break"></p>

Assume:

+ $r_{tgt} = .12$
+ $exp_i = EL_i$ i.e. Fico has no expenses save for loan losses
+ Supported entirely by equity capital

For this exercise, we created a new portfolio via the approach in [loss models](loss.ipynb) with the following properties:

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML, Markdown

from htsfi.main import *

update_style()
plt.style.use('htsfi')

In [2]:
from IPython.core.display import HTML, display, Markdown

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    from htsfi.sims.pricing import *

md = f'+ Number of Loans: {n}'
md += '\n\n'
md += f'+ Total Outstanding: ${EAD.sum():,.0f}'
md += '\n\n'
md += f'+ Expected Loss: ${ld.mean():,.2f}'
md += '\n\n'
md += f'+ Required Equity (99.95% VaR): ${econ_cap:,.2f}'
md += '\n\n'

display(Markdown(md))

+ Number of Loans: 20

+ Total Outstanding: $2,000,000

+ Expected Loss: $25,456.78

+ Required Equity (99.95% VaR): $289,442.94



<p class="p-break"></p>

With that portfolio distribution, we now want to consider how to price 3 loans in the portfolio. The loans have details as follows:

In [3]:
from IPython.core.display import HTML, display

# Loan Portfolio
n = 3
loans_t = 5*10**5
randloans = np.random.uniform(0.1,1,n)
loan_ead = (randloans / randloans.sum())*loans_t
w = loan_ead / loan_ead.sum()

# PD for each borrower/loan.
pd_mu = 0.0044
pd_std = 0.002
p_of_d = np.random.normal(pd_mu, pd_std, n)
count, bins = np.histogram(p_of_d, bins=PDs)
binned = np.digitize(p_of_d, bins)
rats = np.array(SPs)[binned]

p_of_d = np.where(p_of_d<0, 0, p_of_d)

pd_var = bern.var(p_of_d)     # PD for each individual loan is bernoulli distributed and has its own variance.

# LGD
lgd_mu = .4
lgd_std = .4

a, b = beta_params_from_descript(lgd_mu, lgd_std**2)
lgd_freeze = beta(a, b)
lgd = lgd_freeze.rvs(size=n)

lgd_var = np.repeat(lgd.var(), n)

el = loan_ead*p_of_d*lgd
ul = loan_ul(loan_ead, p_of_d, lgd, pd_var, lgd_var)

html = '<table>'
html +=  '<thead>'
html +=    '<tr>'
html +=      '<th class="loan-table"></th>'
html +=      ' '.join([f'<th class="loan-table">Loan {i + 1}</th>' for i in range(n)])
html +=    '</tr>'
html +=  '</thead>'
html +=  '<tbody>'
html +=    '<tr>'
html +=      '<td class="loan-table">Amount</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in loan_ead])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Probability<br>of<br>Default (PD)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in p_of_d])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">S&P<br>Rating</td>'
html +=      ' '.join([f'<td class="loan-table">{val}</td>' for val in rats])
html +=    '</tr>'
html +=    '<tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Loss Given<br>Default (LGD)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in lgd])
html +=    '</tr>'
html +=      r'<td class="loan-table">$\sigma^2_{PD}$</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in pd_var])
html +=    '</tr>'
html +=    '<tr>'
html +=      r'<td class="loan-table">$\sigma^2_{LGD}$</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in lgd_var])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Expected<br>Loss (EL)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in el])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Unexpected<br>Loss (UL)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in ul])
html +=    '</tr>'
html +=  '</tbody>'
html += '</table>'

display(HTML(html))

We can do quick math on each loan to find the interest rate required to meet hurdle:

In [4]:
def calc_ret(rev, exp, cap):
    return (rev - exp) / cap

def int_to_make_hurdle(exp, loan, equity, hurdle):
    return (hurdle*equity + exp) / loan

hurdle = .12
exp = el

debt = 0
equity = loan_ead
req_int = int_to_make_hurdle(exp, loan_ead, equity, hurdle)

In [5]:
from IPython.core.display import HTML, display

html = '<table class="loan-table">'
html +=  '<thead>'
html +=    '<tr>'
html +=      '<th class="loan-table"></th>'
html +=      ' '.join([f'<th class="loan-table">Loan {i + 1}</th>' for i in range(n)])
html +=    '</tr>'
html +=  '</thead>'
html +=  '<tbody>'
html +=    '<tr>'
html +=      '<td class="loan-table">Amount</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in loan_ead])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Probability<br>of<br>Default (PD)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in p_of_d])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Loss Given<br>Default (LGD)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in lgd])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Expected<br>Loss (EL)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in el])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Required<br>Interest Rate</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in req_int])
html +=    '</tr>'
html +=  '</tbody>'
html += '</table>'

display(HTML(html))

,Loan 1,Loan 2,Loan 3
Amount,"230,724.18","158,870.11","110,405.71"
ProbabilityofDefault (PD),0.42%,0.17%,0.47%
Loss GivenDefault (LGD),64.25%,0.34%,0.00%
ExpectedLoss (EL),618.25,0.94,0.00
RequiredInterest Rate,12.27%,12.00%,12.00%


Unsurprisingly, even the lowest risk loan requires at least 12% interest rate. This is way above market for these types of loans.

<p class="p-break"></p>

How then do we get a lower hurdle rate?

1. Greedy-ass investor lowers their expectations

2. Use leverage

<p class="p-break"></p>

**Mix of Debt and Equity**

To show how we can incorporate leverage, we modify our assumptions. For each loan, we will assume that we only need to hold the UL for each loan as equity. The rest of Fico's capital will be debt, borrowed at 3%.

<p class="p-break"></p>

Assume:

+ $r_{tgt} = .12$
+ $E_i = UL_i$
+ $D_i = EAD_i - E_i$
+ $exp_i = EL_i + D_i*r_{debt}$
+ $r_{debt} = .03$

In [6]:
equity = ul
debt = loan_ead - equity

cost_of_debt = .03
int_on_debt = debt * cost_of_debt
exp = el + int_on_debt

req_int = int_to_make_hurdle(exp, loan_ead, equity, hurdle)

In [7]:
from IPython.core.display import HTML, display

html = '<table class="loan-table">'
html +=  '<thead>'
html +=    '<tr>'
html +=      '<th class="loan-table"></th>'
html +=      ' '.join([f'<th class="loan-table">Loan {i + 1}</th>' for i in range(n)])
html +=    '</tr>'
html +=  '</thead>'
html +=  '<tbody>'
html +=    '<tr>'
html +=      '<td class="loan-table">Amount</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in loan_ead])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Probability<br>of<br>Default (PD)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in p_of_d])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Loss Given<br>Default (LGD)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in lgd])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Expected<br>Loss (EL)</td>'
html +=      ' '.join([f'<td class="loan-table">{val:,.2f}</td>' for val in el])
html +=    '</tr>'
html +=    '<tr>'
html +=      '<td class="loan-table">Required<br>Interest Rate</td>'
html +=      ' '.join([f'<td class="loan-table">{val:.2%}</td>' for val in req_int])
html +=    '</tr>'
html +=  '</tbody>'
html += '</table>'

display(HTML(html))

,Loan 1,Loan 2,Loan 3
Amount,"230,724.18","158,870.11","110,405.71"
ProbabilityofDefault (PD),0.42%,0.17%,0.47%
Loss GivenDefault (LGD),64.25%,0.34%,0.00%
ExpectedLoss (EL),618.25,0.94,0.00
RequiredInterest Rate,3.76%,3.12%,3.19%


We can see how the mix of debt and equity has helped to improve loan pricing, which should make Fico more competitive in the market place.

<p class="p-break"></p>


## Market Driven ##

+ TBD

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
%load_ext autoreload
%autoreload 2

from htsfi.main import *

move_to_doc_folder('pricing.ipynb')